In [ ]:
import pandas as pd
import io
import requests
import zipfile
from sklearn.cluster import DBSCAN
import numpy as np


# Load directly from GitHub
url = "https://github.com/clarabot1/LA28_bus_network/raw/main/airbnb_listings.csv.zip"  # Updated URL

# Download the zip file
response = requests.get(url)
response.raise_for_status()  # Check for download errors

# Extract the CSV file from the zip file
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    with zip_ref.open("airbnb_listings.csv") as csv_file:
        df = pd.read_csv(csv_file)

# Show the dataframe
df.head()
airbnb_listings = df

# Data Import

In [ ]:
airbnb_listings.head(1)

In [ ]:
# keep colums that are id, accommodates, latitude, longitude
airbnb_listings = airbnb_listings[["id", "accommodates", "latitude", "longitude"]]
# remove duplicates
airbnb_listings = airbnb_listings.drop_duplicates()
# remove rows with null values
airbnb_listings = airbnb_listings.dropna()
# remove rows with 0 values
airbnb_listings = airbnb_listings[airbnb_listings["accommodates"] > 0]
airbnb_listings.head(5)

In [ ]:
# Load directly from GitHub
url = "https://raw.githubusercontent.com/gr-oll/SLO_LA_Olympics/refs/heads/main/hotels.csv"
df = pd.read_csv(url)

# Show the dataframe
df.head()

hotels = df

In [ ]:
# keep only the relevant columns
hotels = hotels[["id", "people", "lon", "lat"]]
# already handled duplicates and null values
hotels.head(5)

In [ ]:
#accommodates summary statistics
display(airbnb_listings["accommodates"].describe())
#people summary statistics
hotels["people"].describe()

In [ ]:
# Align column names for merging
hotels_renamed = hotels.rename(columns={"lat": "latitude", "lon": "longitude", "people": "accommodates"})

hotels_renamed["id"] = "H" + hotels_renamed["id"].astype(str)
airbnb_listings["id"] = "A" + airbnb_listings["id"].astype(str)

# Merge the dataframes vertically
merged_df = pd.concat([airbnb_listings, hotels_renamed], ignore_index=True)

# Display the merged dataframe
merged_df.head()

In [ ]:
#count the number of listings
len(merged_df)

In [ ]:
#sum of accommodates
merged_df["accommodates"].sum()

# Create clusters

## OG Clusters

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Define the number of clusters (you can adjust this based on your requirements)
num_clusters = 35

# Prepare the data for clustering (latitude, longitude, accommodates)
features_for_clustering = merged_df[["latitude", "longitude", "accommodates"]].copy()

# Normalize the accommodates column to balance its scale with latitude and longitude
scaler = MinMaxScaler()
features_for_clustering["accommodates"] = scaler.fit_transform(features_for_clustering[["accommodates"]])

# Perform KMeans clustering
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
merged_df["kmeans_cluster"] = kmeans.fit_predict(features_for_clustering)

# Analyze the clusters
cluster_summary = merged_df.groupby("kmeans_cluster").agg(
    total_accommodates=("accommodates", "sum"),
    count=("id", "count"),
    avg_latitude=("latitude", "mean"),
    avg_longitude=("longitude", "mean")
)

# Display the cluster summary
cluster_summary

In [ ]:
merged_df

In [ ]:
# Define a color palette for the clusters
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# Generate a color map with distinct colors for each cluster
num_clusters = merged_df["kmeans_cluster"].nunique()
colormap = cm.get_cmap("tab20", num_clusters)
cluster_colors = [colors.rgb2hex(colormap(i)[:3]) for i in range(num_clusters)]

# Create a folium map centered around Los Angeles
m = folium.Map(location=[34.0522, -118.2437], zoom_start=12)

# Map cluster IDs to integer indices
cluster_id_to_index = {cluster_id: idx for idx, cluster_id in enumerate(merged_df["kmeans_cluster"].unique())}

# Add points to the map with colors based on their cluster
for _, row in merged_df.iterrows():
    cluster_index = cluster_id_to_index[row["kmeans_cluster"]]
    cluster_color = cluster_colors[cluster_index]
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5,
        color=cluster_color,
        fill=True,
        fill_color=cluster_color,
        fill_opacity=0.7,
        popup=f"ID: {row['id']}, Accommodates: {row['accommodates']}, Cluster: {row['kmeans_cluster']}"
    ).add_to(m)


# Display the map
m

## Break Down the Big Ones

In [ ]:
def subcluster_large_cluster(cluster_id, n_subclusters, df):
    cluster_df = df[df["kmeans_cluster"] == cluster_id]
    kmeans_sub = KMeans(n_clusters=n_subclusters, random_state=0, n_init=10)
    cluster_df["sub_cluster"] = kmeans_sub.fit_predict(cluster_df[["latitude", "longitude"]])
    df.loc[df["kmeans_cluster"] == cluster_id, "sub_cluster"] = (
        str(cluster_id) + "_" + cluster_df["sub_cluster"].astype(str)
    )
    return df

# Subcluster specified clusters
for cluster_id, n_subclusters in [(0, 4), (9, 6), (17, 3), (21, 4), (25, 4), (29, 2)]:
    if cluster_id in merged_df["kmeans_cluster"].unique():
        merged_df = subcluster_large_cluster(cluster_id, n_subclusters, merged_df)
    else:
        print(f"Cluster {cluster_id} not found.")


merged_df["sub_cluster"] = merged_df["sub_cluster"].fillna(merged_df["kmeans_cluster"].astype(str))
# Print the first few rows to verify
print(merged_df.head())




In [ ]:
merged_df

In [ ]:
#drop cluster 10 - avalon 
merged_df = merged_df[merged_df["kmeans_cluster"] != 10]

In [ ]:
# Analyze the clusters
cluster_summary = merged_df.groupby("sub_cluster").agg(
    total_accommodates=("accommodates", "sum"),
    count=("id", "count"),
    avg_latitude=("latitude", "mean"),
    avg_longitude=("longitude", "mean")
)

# Display the cluster summary
cluster_summary.describe()

In [ ]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# Use a continuous colormap like 'rainbow' to ensure each cluster has a unique color
num_clusters = merged_df["sub_cluster"].nunique()
colormap = cm.get_cmap("rainbow", num_clusters)
cluster_colors = [colors.rgb2hex(colormap(i)[:3]) for i in range(num_clusters)]

m = folium.Map(location=[34.0522, -118.2437], zoom_start=12)
cluster_id_to_index = {cluster_id: idx for idx, cluster_id in enumerate(merged_df["sub_cluster"].unique())}

for _, row in merged_df.iterrows():
    cluster_index = cluster_id_to_index[row["sub_cluster"]]
    cluster_color = cluster_colors[cluster_index]
    folium.CircleMarker(
        location=[row["latitude"], row["longitude"]],
        radius=5,
        color=cluster_color,
        fill=True,
        fill_color=cluster_color,
        fill_opacity=0.7,
        popup=f"ID: {row['id']}, Accommodates: {row['accommodates']}, Cluster: {row['sub_cluster']}"
    ).add_to(m)
for cluster_id, row in cluster_summary.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"Cluster: {cluster_id}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
        icon=folium.Icon(color="blue")
    ).add_to(m)
m


# Accomodations Clusters

In [ ]:
# Create a map centered around Los Angeles
map_center = [34.0522, -118.2437]
cluster_map = folium.Map(location=map_center, zoom_start=10)

# Add markers for each cluster
for cluster_id, row in cluster_summary.iterrows():
    folium.Marker(
        location=[row['avg_latitude'], row['avg_longitude']],
        popup=f"Cluster: {cluster_id}<br>Total Accommodates: {row['total_accommodates']}<br>Count: {row['count']}",
        icon=folium.Icon(color="blue")
    ).add_to(cluster_map)

# Display the map
cluster_map

# Bus Divisions

In [ ]:
#Depot

####Depot dataframe and map

import pandas as pd

# Load directly from GitHub
url = "https://raw.githubusercontent.com/clarabot1/LA28_bus_network/refs/heads/main/bus_divisions_cleaned-2.csv"
df = pd.read_csv(url)

# Show the dataframe
df.head()

bus_divisions = df

# prompt: print the different values in FACILITY column, and drop all lines that are either TERMINAL and LOCATIONS

print(df['FACILITY'].unique())


bus_divisions = bus_divisions[~bus_divisions['FACILITY'].str.contains('LOCATION', na=False)]
bus_divisions = bus_divisions[~bus_divisions['FACILITY'].str.contains('TERMINAL', na=False)]


bus_divisions


In [ ]:

# prompt: #drop rows with NaN in address column in the bus_divisions dataframe, as well as rows with the following 4Digits: 4,6,10,11,20,21,22

# Drop rows with NaN in 'address' column
bus_divisions.dropna(subset=['Address'], inplace=True)

# Define the 4Digits to exclude
four_digits_to_exclude = [4, 6, 10, 11, 12, 20, 21, 22]

# Convert '4Digits' column to numeric, coercing errors to NaN
bus_divisions['4Digits'] = pd.to_numeric(bus_divisions['4Digits'], errors='coerce')

# Drop rows where '4Digits' is in the list of values to exclude
bus_divisions = bus_divisions[~bus_divisions['4Digits'].isin(four_digits_to_exclude)]

#Now you can work with the modified 'bus_divisions' DataFrame
bus_divisions



print(bus_divisions[bus_divisions.isna().any(axis=1)])



# Create a map centered around Los Angeles
map_center = [34.0522, -118.2437]
bus_division_map = folium.Map(location=map_center, zoom_start=10)

# Add markers for each bus division location
for index, row in bus_divisions.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"Bus Division: {row['NAME']}",  # Assuming 'division_name' column exists
        icon=folium.Icon(color="green")
    ).add_to(bus_division_map)

bus_division_map

#### Divisions capacity


In [ ]:


import pandas as pd

# Load directly from GitHub
url = "https://raw.githubusercontent.com/clarabot1/LA28_bus_network/refs/heads/main/bus_divisions_cleaned_capacity.csv"
df = pd.read_csv(url)

# Show the dataframe
df.head()

depot_capacity = df
depot_capacity = depot_capacity.rename(columns={'Amonut': 'amount'})



In [ ]:


# prompt: compute total capacity of each depot, and add the corresponding latitude and longitude from bus_divisions based on the matching 4Digits column

# Merge depot capacity with bus division data based on the '4Digits' column
merged_depot_data = pd.merge(depot_capacity, bus_divisions, on='4Digits', how='left')

# Calculate the total capacity for each depot
merged_depot_data['Total Capacity'] = merged_depot_data.groupby('4Digits')['amount'].transform('sum')

# Select relevant columns and display the result
# The 'Latitude' and 'Longitude' columns were likely renamed during the merge,
# so we need to use their new names.
depot_capacity_with_coordinates = merged_depot_data[['4Digits', 'Total Capacity', 'Latitude_y', 'Longitude_y']].drop_duplicates()
depot_capacity_with_coordinates = depot_capacity_with_coordinates.rename(columns={'Latitude_y': 'Latitude', 'Longitude_y': 'Longitude'})
depot_capacity_with_coordinates

In [ ]:
# sum total capacity
total_capacity = depot_capacity_with_coordinates['Total Capacity'].sum()
total_capacity

# Olympic venues

In [ ]:

####Dataframe and map

import pandas as pd

# Load directly from GitHub
url = "https://raw.githubusercontent.com/clarabot1/LA28_bus_network/refs/heads/main/Venue_data-4.csv"
df = pd.read_csv(url)

# Show the dataframe
df.head()

venues_locations = df

venues_locations


In [ ]:

map_center = [34.0522, -118.2437]
la_map_venues = folium.Map(location=map_center, zoom_start=10)

# Add markers for each venue location
for index, row in venues_locations.iterrows():
    try:
        latitude = float(row['Latitude'])
        longitude = float(row['Longitude'])
        folium.Marker(
            location=[latitude, longitude],
            popup=row['Venue'],  # Assuming 'Venue' column exists
            icon=folium.Icon(color="purple")
        ).add_to(la_map_venues)
    except (ValueError, TypeError):
        print(f"Skipping row {index} due to invalid latitude/longitude values.")
    except KeyError:
      print(f"Skipping row {index} due to missing 'Venue' column")

la_map_venues



In [ ]:

####Capacity

# prompt: create a copy of venues_locations called venues locations capacity, and only return venue, latitude, longitude and capacity

venues_locations_capacity = venues_locations[['Venue', 'Latitude', 'Longitude', 'Approx. Capacity']].copy()
venues_locations_capacity

# Distance Matrix

In [ ]:
#reset the index of the cluster summary dataframe
cluster_summary = cluster_summary.reset_index()
#drop the sub_cluster column
cluster_summary = cluster_summary.drop(columns=['sub_cluster'])
cluster_summary


In [ ]:
# add a column to the venues_locations_capacity with an id Starting with V
venues_locations_capacity['id'] = 'V' + (venues_locations_capacity.index + 1).astype(str)
# add a column to the cluster summary with an id Starting with C
cluster_summary['id'] = 'A' + (cluster_summary.index + 1).astype(str)
display(venues_locations_capacity.head(5))
cluster_summary.head(5)

In [ ]:
#in venues_locations_capacity, rename avg with latitude and longitude
venues_locations_capacity = venues_locations_capacity.rename(columns={'avg_latitude': 'Latitude', 'avg_longitude': 'Longitude'})
venues_locations_capacity

In [ ]:
# Select relevant columns from both dataframes
venues_data = venues_locations_capacity[['Latitude', 'Longitude', 'id']]
clusters_data = cluster_summary[['avg_latitude', 'avg_longitude', 'id']].rename(
    columns={'avg_latitude': 'Latitude', 'avg_longitude': 'Longitude'}
)

# Concatenate the dataframes vertically
df_matrix = pd.concat([venues_data, clusters_data], ignore_index=True)

# Display the merged dataframe
df_matrix.head()

In [ ]:
import googlemaps

# Replace "YOUR_API_KEY" with your actual Google Maps API key
gmaps = googlemaps.Client(key="API_KEY")

# Function to calculate distance matrix
def calculate_distance_matrix(origins, destinations):
    distance_matrix = gmaps.distance_matrix(origins, destinations, mode="driving")
    return distance_matrix

# Example usage
origins = df_matrix['id'].tolist()
destinations = df_matrix['id'].tolist()


# Assuming df_matrix DataFrame has columns 'Latitude' and 'Longitude'
origin_coords = df_matrix[['Latitude', 'Longitude']].values.tolist()
dest_coords = df_matrix[['Latitude', 'Longitude']].values.tolist()

distance_matrix_data = []
for i in range(len(origins)):
  try:
    distance_matrix_row = []
    for j in range(len(destinations)):
        # Access the origin and destination coordinates as tuples
        origin_coord = tuple(origin_coords[i])  # Convert to tuple
        dest_coord = tuple(dest_coords[j])  # Convert to tuple

        distance_result = gmaps.distance_matrix(origin_coord, dest_coord, mode="driving")
        if distance_result['rows'][0]['elements'][0]['status'] != 'OK':
          distance_matrix_row.append(np.nan)
        else:
          distance_matrix_row.append(distance_result['rows'][0]['elements'][0]['distance']['value'])
    distance_matrix_data.append(distance_matrix_row)
  except Exception as e:
    print(f"Error calculating distance for row {i}: {e}")
    distance_matrix_data.append([np.nan] * len(destinations))


distance_matrix_df = pd.DataFrame(distance_matrix_data, index = origins, columns = destinations)
##distances in meters


In [ ]:
# Assuming df_matrix DataFrame has columns 'Latitude' and 'Longitude'
origin_coords = df_matrix[['Latitude', 'Longitude']].values.tolist()
dest_coords = df_matrix[['Latitude', 'Longitude']].values.tolist()
origins = df_matrix['id'].tolist()
destinations = df_matrix['id'].tolist()

time_matrix_data = []
for i in range(len(origins)):
  try:
    time_matrix_row = []
    for j in range(len(destinations)):
        origin_coord = tuple(origin_coords[i])  # Convert to tuple
        dest_coord = tuple(dest_coords[j])  # Convert to tuple

        distance_result = gmaps.distance_matrix(origin_coord, dest_coord, mode="driving")
        if distance_result['rows'][0]['elements'][0]['status'] != 'OK':
          time_matrix_row.append(np.nan)
        else:
          time_matrix_row.append(distance_result['rows'][0]['elements'][0]['duration']['value'])
    time_matrix_data.append(time_matrix_row)
  except Exception as e:
    print(f"Error calculating time for row {i}: {e}")
    time_matrix_data.append([np.nan] * len(destinations))

time_matrix_df = pd.DataFrame(time_matrix_data, index = origins, columns = destinations)
# times in seconds

time_matrix_df

In [ ]:
#export the distance matrix to a csv file
distance_matrix_df.to_csv("distance_matrix.csv")
#export the time matrix to a csv file
time_matrix_df.to_csv("time_matrix.csv")

In [ ]:
#export venues_locations_capacity to a csv file
venues_locations_capacity.to_csv("venues.csv", index=False)
#export cluster_summary to a csv file
cluster_summary.to_csv("clusters_central_location.csv", index=False)
#export merged_df to a csv file
merged_df.to_csv("accomodations.csv", index=False)

In [ ]:
import googlemaps
from datetime import datetime

# Initialize the Google Maps client with your API key
gmaps = googlemaps.Client(key='KEY')

# Build list of origin and destination tuples using the dataframe coordinates
origins = accomodations_clusters[['avg_latitude', 'avg_longitude']].apply(lambda row: (row['avg_latitude'], row['avg_longitude']), axis=1).tolist()
destinations = bus_terminals[['Latitude', 'Longitude']].apply(lambda row: (row['Latitude'], row['Longitude']), axis=1).tolist()

# Request the distance matrix in batches to avoid MAX_DIMENSIONS_EXCEEDED error
max_elements = 100
max_destinations_per_request = 25  # Free API limit for destinations per request
destination_batch_size = max_destinations_per_request
origin_batch_size = max_elements // destination_batch_size

# Initialize an empty results matrix
results = [[None] * len(destinations) for _ in range(len(origins))]

for i in range(0, len(origins), origin_batch_size):
    origin_chunk = origins[i:i+origin_batch_size]
    for j in range(0, len(destinations), destination_batch_size):
        destination_chunk = destinations[j:j+destination_batch_size]
        response = gmaps.distance_matrix(origin_chunk, destination_chunk, mode='driving')
        for index_in_chunk, row in enumerate(response['rows']):
            result_row = i + index_in_chunk
            for dest_index, element in enumerate(row['elements']):
                if element['status'] == 'OK':
                    results[result_row][j+dest_index] = element['duration']['value']
                else:
                    results[result_row][j+dest_index] = None

time_matrix = pd.DataFrame(results, index=accomodations_clusters['id'], columns=bus_terminals['id'])
time_matrix

id,BT25,BT07,BT13,BT10,BD01,BD02,BD05,BD06,BD09,BD11,...,BL22,BL23,BT26,BL19,BT04,BT19,BL03,BL07,BT05,BT15
id,,,,,,,,,,,,,,,,,,,,,
A1,2234,1766,1370,2384,2077,1938,1932,2472,1853,2021,...,2529,2558,1594,1861,1854,1850,2308,1811,1372,2128
A2,1750,1294,1042,1912,1632,1787,2402,1988,1382,1930,...,2580,2314,1486,1685,1472,1664,1818,1758,954,1717
A3,1254,779,948,1397,1116,1271,2086,1594,866,1415,...,2265,1798,1356,1170,957,1149,1302,1243,341,1201
A4,1741,1267,650,1885,1604,1726,2050,2082,1354,1809,...,2647,2346,1119,1700,1539,1638,1790,1599,814,1689
A5,2270,1640,2254,2414,1681,1453,2260,2348,1561,1614,...,2979,610,2077,1731,1724,1255,1826,1442,2052,1394
A6,2628,1979,2593,2693,2022,1839,2873,2634,1900,2000,...,3592,1144,2463,2116,2110,1641,2178,1828,2391,1780
A7,1489,1069,1479,1632,1161,786,1466,1764,1042,716,...,2166,1518,1260,949,943,830,1306,773,1343,1135
A8,1230,1933,2268,1603,2163,2282,2980,1108,1913,2350,...,3159,2813,2353,1974,1966,2194,2232,2140,1774,2229
A9,3111,2902,2604,3466,2995,2619,1744,3349,2875,2658,...,2059,3352,2615,2783,2776,2663,3140,2606,2648,2969


In [ ]:
import googlemaps
from datetime import datetime

# Initialize the Google Maps client with your API key
gmaps = googlemaps.Client(key='Key')

# Build list of origin and destination tuples using the dataframe coordinates
origins = venues[['Latitude', 'Longitude']].apply(lambda row: (row['Latitude'], row['Longitude']), axis=1).tolist()
destinations = bus_terminals[['Latitude', 'Longitude']].apply(lambda row: (row['Latitude'], row['Longitude']), axis=1).tolist()

# Request the distance matrix in batches to avoid MAX_DIMENSIONS_EXCEEDED error
max_elements = 100
max_destinations_per_request = 25  # Free API limit for destinations per request
destination_batch_size = max_destinations_per_request
origin_batch_size = max_elements // destination_batch_size

# Initialize an empty results matrix
results = [[None] * len(destinations) for _ in range(len(origins))]

for i in range(0, len(origins), origin_batch_size):
    origin_chunk = origins[i:i+origin_batch_size]
    for j in range(0, len(destinations), destination_batch_size):
        destination_chunk = destinations[j:j+destination_batch_size]
        response = gmaps.distance_matrix(origin_chunk, destination_chunk, mode='driving')
        for index_in_chunk, row in enumerate(response['rows']):
            result_row = i + index_in_chunk
            for dest_index, element in enumerate(row['elements']):
                if element['status'] == 'OK':
                    results[result_row][j+dest_index] = element['duration']['value']
                else:
                    results[result_row][j+dest_index] = None

venues_to_bus_matrix = pd.DataFrame(results, index=venues['id'], columns=bus_terminals['id'])
venues_to_bus_matrix

/Users/leonardogreco/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


id,BT25,BT07,BT13,BT10,BD01,BD02,BD05,BD06,BD09,BD11,...,BL22,BL23,BT26,BL19,BT04,BT19,BL03,BL07,BT05,BT15
id,,,,,,,,,,,,,,,,,,,,,
V1,636.0,526.0,1008.0,780.0,772.0,890.0,2022.0,912.0,521.0,958.0,...,2695.0,1511.0,961.0,582.0,638.0,802.0,980.0,749.0,806.0,856.0
V2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
V3,2150.0,2270.0,2883.0,2356.0,2312.0,2268.0,3587.0,1883.0,2191.0,2514.0,...,4287.0,2326.0,2882.0,2557.0,2498.0,2185.0,2472.0,2343.0,2681.0,1968.0
V4,772.0,343.0,831.0,916.0,639.0,587.0,1718.0,1048.0,405.0,654.0,...,2391.0,1207.0,615.0,307.0,217.0,498.0,841.0,445.0,617.0,740.0
V5,946.0,1498.0,1797.0,1121.0,1729.0,1848.0,2575.0,1293.0,1479.0,1916.0,...,2754.0,2468.0,1919.0,1539.0,1532.0,1760.0,1797.0,1706.0,1359.0,1814.0
V6,1545.0,2036.0,2649.0,1784.0,2079.0,2045.0,3348.0,1318.0,1957.0,2276.0,...,3992.0,2305.0,2609.0,2230.0,2222.0,1916.0,1912.0,2104.0,2448.0,1721.0
V7,1783.0,2345.0,2912.0,2115.0,2388.0,2354.0,3658.0,1491.0,2266.0,2585.0,...,4165.0,2615.0,2865.0,2486.0,2478.0,2226.0,2222.0,2413.0,2710.0,2030.0
V8,772.0,343.0,831.0,916.0,639.0,587.0,1718.0,1048.0,405.0,654.0,...,2391.0,1207.0,615.0,307.0,217.0,498.0,841.0,445.0,617.0,740.0
V9,1390.0,1881.0,2495.0,1629.0,1924.0,1890.0,3194.0,1163.0,1802.0,2121.0,...,3837.0,2151.0,2454.0,2075.0,2067.0,1762.0,1758.0,1949.0,2293.0,1566.0
